In [1]:
using PyCall
unshift!(PyVector(pyimport("sys")["path"]), "");
@pyimport GLS_Jul_weekday_PM_ext

No dicts found; please check load_dicts...
Optimize a model with 462 rows, 462 columns and 462 nonzeros
Model has 106953 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [8e-02, 1e+05]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 462 rows and 0 columns
Presolve time: 0.01s
Presolved: 0 rows, 462 columns, 0 nonzeros
Presolved model has 106953 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 64
 AA' NZ     : 2.016e+03
 Factor NZ  : 2.080e+03
 Factor Ops : 8.944e+04 (less than 1 second per iteration)
 Threads    : 4

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -7.98688753e+07  0.00000000e+00  2.77e+05 1.63e+02  1.00e+06     0s
   1  -2.34315639e+06 -2.40472712e+06  1.47e+04 8.64e+00  6.24e+04     0s
   2   4.90631654e+05 -1.00152249e+06  7.50e+02 4.41e-01  6.14e+03     0s
   3   9.

In [2]:
xi_list = GLS_Jul_weekday_PM_ext.xi_list
P = GLS_Jul_weekday_PM_ext.P
L = GLS_Jul_weekday_PM_ext.L

462

In [3]:
size(P, 1), size(P, 2)

(462,8260)

In [4]:
fictitious_OD_list = GLS_Jul_weekday_PM_ext.zload("../temp_files/fictitious_OD_list")

190-element Array{Any,1}:
 375
 331
 409
 438
 404
 443
  61
 429
 207
 353
 439
 206
 392
   ⋮
 393
  16
 366
 450
 142
 422
  58
 269
 364
 379
 188
 436

In [5]:
demandsSmall = GLS_Jul_weekday_PM_ext.demandsSmall
odMap = GLS_Jul_weekday_PM_ext.odMap
odIdxExt = GLS_Jul_weekday_PM_ext.odIdxExt

56-element Array{Any,1}:
   2
   5
  10
  11
  12
  14
  15
  63
  68
  73
  74
  75
  77
   ⋮
 321
 326
 327
 328
 330
 336
 339
 342
 347
 348
 349
 351

In [ ]:
using JuMP

mGLSJulia = Model()

@defVar(mGLSJulia, lam[1:size(P,1)] >= 0)

# for l in fictitious_OD_list
#     @addConstraint(mGLSJulia, lam[l] == 0)
# end

# for j = 1:length(odMap)
#     @addConstraint(mGLSJulia, lam[odIdxExt[j]] - demandsSmall["$(j-1)"] <= 0.2 * demandsSmall["$(j-1)"])
#     @addConstraint(mGLSJulia, demandsSmall["$(j-1)"] - lam[odIdxExt[j]] <= 0.2 * demandsSmall["$(j-1)"])
# end

@defVar(mGLSJulia, p[1:size(P,1), 1:size(P,2)] >= 0)

for i = 1:size(P,1)
    for j = 1:size(P,2)
        if P[i,j] == 0
            @addConstraint(mGLSJulia, p[i,j] == 0)
        end
    end
end
            
for i = 1:size(P,1)
    @addNLConstraint(mGLSJulia, sum{p[i,j], j = 1:size(P,2)} == 1)
end

for l = 1:L
    @addNLConstraint(mGLSJulia, sum{p[i,l] * lam[i], i = 1:size(P,1)} == xi_list[l])
end
    
@setNLObjective(mGLSJulia, Min, sum{p[1,j], j = 1:size(P,2)})  # play no actual role, but could not use zero objective

solve(mGLSJulia)